In [ ]:
from graph_utils import pose_graph_nx as pg
from graph_utils import pose_graph_helper as helper
from graph_utils import plotter as plt

import pandas as pd
import numpy as np
import glob, os
import copy
import pickle

%load_ext autoreload
%autoreload 2

## Load a graph
Firstly, load any pose graph to test the sparsification on. It can be load directly from the database, or an unpickled PoseGraph object.

In [ ]:
graph = pg.PoseGraph()
graph.loadTripFromDb(1675315172)


### Graph porperties
These are the properties of the original, dense, graph.

In [ ]:
graph.calculateEdgeDistances()
graph_helper = helper.PoseGraphHelper(graph)

graph_helper.printGraphProperties()

### Plot loaded graph
And this is what the original graph looks like.

In [ ]:
plotter = plt.Plotter()
plotter.addNodeLayer(graph, color=(255, 0, 0, 1))

plotter.plotGmaps()

### Prune graph by merging parallel paths
tbd

### Sparsify graph by dropping all nodes that are no points of interest
The most radical application is to drop all nodes that are no intersections or door events. This will result in a high level representation of the graph.

In [ ]:
from graph_utils import sparsifier as sparse
import copy

graph_sparse = sparse.sparsifiyPoseGraph(copy.deepcopy(graph), 
                                         criteria='PoI'
                                        )

### Graph properties
The high level graph has the following properties. 
The subsequent sparsification methods should under no circumstances fall below these properties, otherwise valuable information would be lost. 

In [ ]:
graph_sparse.calculateEdgeDistances()
graph_sparse_helper = helper.PoseGraphHelper(graph_sparse)
graph_sparse_helper.printGraphProperties()

### Plot sparsified graph
Add the high level graph (in blue) to the dense graph.

In [ ]:
plotter.addNodeLayer(graph_sparse, color=(0, 0, 255, 1))
plotter.addEdgeLayer(graph_sparse, color=(0, 0, 255, 1))

plotter.plotGmaps()

## Compute L2 loss between pruned and original graph

In [ ]:
import networkx as nx
import math

def difference(pg_dense, pg_sparse):

    diff_edges = pg_dense.edges - pg_sparse.edges
    
    d_edges = iter(pg_dense.edges)
    s_edges = iter(pg_sparse.edges)

    loss = 0
    i=0
    
    s_edge=next(s_edges, None)
    d_edge=next(d_edges, None)
    
    while s_edge is not None:
                
        s_edge_vector = [pg_sparse.nodes[s_edge[1]]['x'] - pg_sparse.nodes[s_edge[0]]['x'],
                         pg_sparse.nodes[s_edge[1]]['y'] - pg_sparse.nodes[s_edge[0]]['y']]
        
            
        while d_edge is not None:
            if s_edge==d_edge:
                d_edge = next(d_edges, None)
                break
                
            if d_edge in diff_edges:
                i = i+1

                d_edge_vector = [pg_dense.nodes[d_edge[1]]['x'] - pg_sparse.nodes[s_edge[0]]['x'],
                                 pg_dense.nodes[d_edge[1]]['y'] - pg_sparse.nodes[s_edge[0]]['y']]
                
                length = np.linalg.norm(d_edge_vector)

                if length != 0 and np.linalg.norm(s_edge_vector)!=0.0:
                    angle = sparse.angle_between(s_edge_vector, d_edge_vector)
                    delta_loss = (math.sin(angle)*length)**2
                    loss += 0.5*(math.sin(angle)*length)**2

            d_edge = next(d_edges, None)

        s_edge = next(s_edges, None)

    return loss / i

In [ ]:
print(difference(graph.nx_graph, graph_sparse.nx_graph))

## Linear fit - grid search hyperparameter

In [ ]:
for window_size in [3,5,10]:
    for loss_threshold in [1, 5, 10]:
        print("\n\n\n Window size = {0}; loss threshold = {1}".format(window_size, loss_threshold))
        
        graph_sparse = sparse.sparsifiyPoseGraph(copy.deepcopy(graph), 
                                                 window_size=window_size, 
                                                 loss_threshold=loss_threshold,
                                                 criteria='fit'
                                                )
        
        # Evaluation
        helper.PoseGraphHelper(graph_sparse).printGraphProperties()
        
        print(difference(graph.nx_graph, graph_sparse.nx_graph))

Why are all path lengths the same. Check implementation in pose_graph_nx serial node drop

## Heading cluster - grid search hyperparameter

In [ ]:
for window_size in [3,5,10]:
    for angle_var_threshold in [0.01,0.05,0.1]:
        print("\n\n\n Window size = {0}; Angel_var_threshold = {1}".format(window_size, angle_var_threshold))
        
        graph_sparse = sparse.sparsifiyPoseGraph(copy.deepcopy(graph), 
                                                 window_size=window_size, 
                                                 loss_threshold=angle_var_threshold,
                                                 criteria='heading'
                                                )
        
        # Evaluation
        graph_sparse.calculateEdgeDistances()
        graph_sparse_helper = helper.PoseGraphHelper(graph_sparse)
        graph_sparse_helper.printGraphProperties()
        
        print(difference(graph.nx_graph, graph_sparse.nx_graph))